In [353]:
import time
import re

In [354]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [355]:
wd = "D:\\Users\\kimdu\\Desktop\\GRAD\\chromedriver.exe"
addr = "https://news.nate.com/view/20200225n34937?mid=n1008"    # 기사 주소 넣기
addr_nums = re.findall("\d+", addr)
addr_comment = 'http://comm.news.nate.com/Comment/ArticleComment/List?artc_sq='+addr_nums[0]+'n'+addr_nums[1] # 댓글창 주소

In [356]:
driver = webdriver.Chrome(wd)
driver.get(addr_comment)

In [357]:
pages = 2

In [358]:
try:
    while True:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"body > div.commentBox.reply_hide > div:nth-child(3) > div.paging_wrap > a:nth-child("+str(pages)+")"))).click()
        time.sleep(2)
        print(pages, end=" ")
        pages+=1
except exceptions.ElementNotVisibleException as e: # 페이지 끝
    pass
except Exception as e: # 다른 예외 발생시 확인
    print(e)

2345678910Message: 



In [359]:
html = driver.page_source
dom = BeautifulSoup(html, "lxml")

In [360]:
# 댓글이 들어있는 페이지 전체 크롤링
comments_raw = dom.find_all('dd',{'class':'usertxt'})
# 댓글의 text만 뽑는다.
comments = [comment.text for comment in comments_raw]


In [361]:
comments[:3]

['\n\t\t\t유느님\t\t\t ',
 '\n늘  앞선 솔선수범선행 \n멋져요\t\t\t ',
 '\n정말 멋지십니다~~~\n재벌들은 가만있는데 정말 대단하세요~~~👍👍👍👍\t\t\t ']

In [362]:
p_comment_list = []

In [363]:
for comment in comments:
    p_comment_list.append(comment.strip())

In [364]:
file = open('p_comment44.txt','w',encoding = 'utf-8')
for comment in p_comment_list:
    file.write(comment + '\n')
file.close()